In [1]:
import datetime
import numpy as np
import pandas as pd
import requests
import random
import time
import re
import jieba as jb
from bs4 import BeautifulSoup
import bs4

In [2]:
def get_ppt_page(url):
    # 紀錄cookies 是否年滿18歲
    resp = requests.get(
        url=url,
        cookies={'over18': '1'}  
    )
    if resp.status_code != 200:
        print('Invalid url:', resp.url)
        return None
    else:
        return resp.text

In [3]:
def get_content(article_url):
    # 文章連結
    # URL = "https://www.ptt.cc/bbs/Gossiping/M.1639675838.A.A33.html"
    # 設定Header與Cookie
    my_headers = {'cookie': 'over18=1;'}
    # 發送get 請求 到 ptt 八卦版
    response = requests.get(article_url, headers = my_headers)


    #  把網頁程式碼(HTML) 丟入 bs4模組分析
    soup = bs4.BeautifulSoup(response.text,"html.parser")

    ## PTT 上方4個欄位
    header = soup.find_all('span','article-meta-value')


    ## 查找所有html 元素 抓出內容
    main_container = soup.find(id='main-container')
    # 把所有文字都抓出來
    all_text = main_container.text
    # 把整個內容切割透過 "-- " 切割成2個陣列
    pre_text = all_text.split('--')[0]
        
    # 把每段文字 根據 '\n' 切開
    texts = pre_text.split('\n')
    # 如果你爬多篇你會發現 
    contents = texts[2:]
    # 內容
    content = '\n'.join(contents)

    # 顯示
    #print('內容: ', content)
    return content

In [4]:
def get_pageinfo(num, resdata, domain_url, pptdata, date_range):
    soup = BeautifulSoup(resdata, 'html5lib')
    # 取得上一頁按鈕
    paging_div = soup.find('div', 'btn-group btn-group-paging')
    prev_url = paging_div.find_all('a')[1]['href']
    
    # 儲存取得的文章資料<div class="r-ent"></div>
    date_divs = soup.find_all('div', 'r-ent')
    
    whether_prev = True
    for k in date_divs:
        # 判斷文章是否在一週內發佈
        week_date = k.find('div', 'date').text.strip() in date_range
        
        # 代表最早的日期，就不符合條件，所以不用下一頁
        # 因為第一頁涵蓋熱門文章，因此日期並非按照順序，所以從第二頁開始判斷
        if num >= 1:
            if week_date == False:
                whether_prev = False

        # 一周內發布       
        if week_date:
            # 發布日期
            post_date = k.find('div', 'date').text.strip()
            #print("發布日期",post_date)
            
            # 推文數
            push_count = k.find('div', 'nrec').text
            push_num = 0
            if push_count:
                try:
                    push_num = int(push_count)  
                except ValueError:
                    # 若轉換失敗，可能是'爆'或 'X1', 'X2'
                    if push_count == '爆':
                        push_num = 100
                    elif push_count.startswith('X'):
                        push_num = -100
                    else:
                        push_num = 0
            #print("推文數",push_num)
            
            # 有超連結，代表文章存在
            if k.find('a'): 
                # 文章標題
                title = k.find('a').text
                #print("標題",title)
                # 文章連結
                href = k.find('a')['href']
                #print("標題連結",domain_url+href)
                # 文章內文
                content = get_content(domain_url+href)

                # 儲存資料
                pptdata.append({
                    'date': post_date,
                    'push_num': push_num,
                    'title': title,
                    'content': content,
                    'href': domain_url+href
                })

        
    
    # 每做完一頁，num+1
    num += 1
    
    
    return prev_url, pptdata, whether_prev, num

In [5]:
if __name__ == '__main__':
    # *** 選擇八卦版 ***
    domain_url = 'https://www.ptt.cc'
    Gossiping = '/bbs/Gossiping/index.html'
    today = time.strftime("%m/%d").lstrip('0')
    date_range = [today]
    pptdata = []  
    num = 0

    # 第一頁 (含熱門文章)
    web_url = domain_url + Gossiping
    ppt_page = get_ppt_page(web_url)
    prev_href, pptdata, whether_prev, num = get_pageinfo(num, ppt_page, domain_url, pptdata, date_range) 
    
    # 往前一頁
    while whether_prev:
        time.sleep(3)
        web_url = domain_url+prev_href
        ppt_page = get_ppt_page(web_url)
        prev_href, pptdata, whether_prev, num = get_pageinfo(num, ppt_page, domain_url, pptdata, date_range)

    # 將list轉成dataframe
    df = pd.DataFrame(pptdata, columns=['date','push_num', 'title', 'content', 'href'])  

    # 存成dataframe
      

In [6]:
date_ = date_range[0].replace("/", "_")

In [7]:
df.to_csv(date_+"_ppt_result.csv", index=False)

In [9]:
df

,date,push_num,title,content,href
0,12/25,2,[問卦] 「國家機器」到底是指什麼？,之前在選舉時，某總機經常被抹黑\n指出國家機器動搖的很厲害\n我其實當下聽到這個詞覺得很莫名...,https://www.ptt.cc/bbs/Gossiping/M.1640436629....
1,12/25,1,Re: [問卦] 民視的歌唱節目是發生什麼事？,很難理解嗎 這不就台灣有資源老人認為現在年輕人應該有的樣子的投射嗎\n因為我年輕的時候這樣子...,https://www.ptt.cc/bbs/Gossiping/M.1640436700....
2,12/25,2,[新聞] 日本不派官員參加北京冬奧會 公布創紀錄,1.媒體來源:新唐人\n\n2.記者署名:趙鳳華 姜迪亞\n\n3.完整新聞標題:\n日本不...,https://www.ptt.cc/bbs/Gossiping/M.1640436756....
3,12/25,7,[問卦] 運動場上黃猴子只能被白人、黑人屌虐？,https://youtu.be/RUDSnH6l6JY?t=478\n\n運動場上，白人、...,https://www.ptt.cc/bbs/Gossiping/M.1640436791....
4,12/25,4,[問卦] 酬神戲放 A 片可以嗎？,\n小時候住鄉下有個習俗\n\n由於拜神的人很多 向神許願的也不少\n\n一但如願了 多多少...,https://www.ptt.cc/bbs/Gossiping/M.1640436820....
...,...,...,...,...,...
1346,12/25,56,[新聞] 上兵站哨睡覺被送法辦 士林地檢聲請簡易,中央社\n（中央社記者蕭博文台北24日電）\n\n上兵站哨睡覺被送法辦 士林地檢聲請簡易判決...,https://www.ptt.cc/bbs/Gossiping/M.1640361673....
1347,12/25,2,[問卦] 小七素食484很不錯？,\n小七素食484很不錯\n滿好吃的\n好像叫天素地蔬\n炒飯 義大利麵 燉飯 三...,https://www.ptt.cc/bbs/Gossiping/M.1640361694....
1348,12/25,0,[問卦] 有人自己尻了嗎？,今年聖誕節 商人炒作出來的日子\n\n雖然是這樣說但我好羨慕街道上的情侶喔\n\n每個都嗯嗯...,https://www.ptt.cc/bbs/Gossiping/M.1640361726....
1349,12/25,57,[問卦] 中國在衰落了嗎？,如題\n\n有在追的幾個時事評論YT\n\n最近幾乎統一口徑在唱衰中國\n\n感覺最近中共的...,https://www.ptt.cc/bbs/Gossiping/M.1640361803....
